In [89]:
import pandas as pd
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


import tensorflow 
import seaborn as sns

from tensorflow import keras
from keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

import cv2

import numpy as np


from sklearn.linear_model import LinearRegression

In [90]:
# Definir la ruta del dataset
DATASET_PATH = r'C:\Users\alber\OneDrive\Escritorio\asl_dataset'

In [91]:
# Obtener las clases disponibles (carpetas dentro del dataset)
clases = sorted(os.listdir(DATASET_PATH))  # ['0', '1', ..., '9', 'A', 'B', ..., 'Z']
num_clases = len(clases)

X = []
y = []

# Cargar imágenes y convertirlas a arrays
for label, clase in enumerate(clases):
    folder_path = os.path.join(DATASET_PATH, clase)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convertimos a escala de grises
        img = cv2.resize(img, (64, 64))  # Redimensionamos a 64x64 píxeles
        X.append(img.flatten())  # Convertimos la imagen en un vector
        y.append(label)  # Guardamos la etiqueta de la clase

# Convertir a arrays de numpy
X = np.array(X)
y = np.array(y)

# Mezclar los datos aleatoriamente
indices = np.arange(len(X))
np.random.shuffle(indices)

# Aplicar el desorden a X e y
X = X[indices]
y = y[indices]

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Guardar los datos para uso futuro
np.save("asl_features_train.npy", X_train)
np.save("asl_labels_train.npy", y_train)
np.save("asl_features_test.npy", X_test)
np.save("asl_labels_test.npy", y_test)

print(f"Datos cargados y mezclados: {X.shape[0]} imágenes, {num_clases} clases")
print(f"Train: {X_train.shape[0]} imágenes | Test: {X_test.shape[0]} imágenes")


Datos cargados y mezclados: 2515 imágenes, 36 clases
Train: 2012 imágenes | Test: 503 imágenes


In [92]:
X_train.shape

(2012, 4096)

In [93]:
X_test.shape

(503, 4096)

In [94]:
lr = LinearRegression()

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [96]:
lasso = linear_model.Lasso()


pipe = Pipeline([('Linear', LinearRegression())])

pipe2 = Pipeline([('Standard Scaler', StandardScaler())])

In [97]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def ejecutar(pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train) 
    pred = pipe.predict(X_test)  
    
    # Calcula y muestra el error cuadrático medio
    mse = mean_squared_error(y_test, pred)
    print(f"MSE = {mse}")

    # Validación cruzada usando el propio pipeline
    cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    print(f"Cross-validation MSE: {-cv_scores.mean()}") 


In [98]:
def ejecutar2(pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train)
    
    # Mostrar solo una parte de X_train para evitar impresión masiva
    if hasattr(X_train, "head"):  # Si es un DataFrame de pandas
        print(X_train.head())
    else:  # Si es un array de NumPy
        print(X_train[:5])


In [99]:
ejecutar(pipe, X_train, X_test, y_train, y_test)

MSE = 63.59224539431879
Cross-validation MSE: 58.98177189811038


In [100]:
ejecutar2(pipe2, X_train, X_test, y_train, y_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [101]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('Standard Scaler', StandardScaler())])

In [102]:
pipe

Pipeline(steps=[('Linear', LinearRegression())])

In [104]:
model = keras.Sequential([ 
    keras.layers.Dense(128, activation='relu'),  
    keras.layers.Dense(64, activation='relu'),   
    keras.layers.Dense(36, activation='softmax') # Capa de salida con 10 neuronas (10 clases)
])

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [106]:
pipeline = Pipeline([('model', model)])

In [107]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
             )


In [108]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


model.fit(X_train, y_train, epochs=36, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.1293 - loss: 44.4026 - val_accuracy: 0.2008 - val_loss: 2.9975
Epoch 2/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2031 - loss: 3.0237 - val_accuracy: 0.2326 - val_loss: 2.8652
Epoch 3/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2415 - loss: 2.8334 - val_accuracy: 0.2763 - val_loss: 2.7822
Epoch 4/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2889 - loss: 2.6917 - val_accuracy: 0.2584 - val_loss: 2.7952
Epoch 5/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3179 - loss: 2.4951 - val_accuracy: 0.3718 - val_loss: 2.1927
Epoch 6/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4078 - loss: 2.2140 - val_accuracy: 0.3718 - val_loss: 2.1779
Epoch 7/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3929 - loss: 2.2477 - val_accuracy: 0.3559 - val_loss: 2.3061
Epoch 8/36
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4016 - loss: 2.1605 - val_accuracy: 0.3837 - val_

In [109]:

from sklearn.cluster import KMeans

# Ejemplo de KMeans con 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
labels = kmeans.predict(X)


In [110]:

from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder con manejo de categorías nuevas en test
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)


MemoryError: Unable to allocate 8.41 GiB for an array with shape (2012, 561311) and data type float64

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Definir modelo y parámetros
model = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

# GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_
print("Mejores parámetros:", grid_search.best_params_)


Mejores parámetros: {'max_depth': None, 'n_estimators': 200}


In [ ]:

# Para asegurar la reproducibilidad del código, usa un entorno virtual.
# Ejecuta esto en la terminal:

# Crear entorno virtual
# python -m venv env

# Activar (Windows)
# env\Scripts\activate

# Activar (Mac/Linux)
# source env/bin/activate

# Guardar dependencias
# pip freeze > requirements.txt

# Agregar `requirements.txt` y `.gitignore` al repositorio para evitar archivos innecesarios.


In [ ]:

from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
